In [37]:
%matplotlib inline
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np
import aruco
import sqlite3
from sqlite3 import Error

In [38]:
CV_CAP_PROP_POS_MSEC = 0
CV_CAP_PROP_FRAME_COUNT = 7

In [39]:
# load board and camera parameters
camparam = aruco.CameraParameters()
camparam.readFromXMLFile("dfk72_6mm_param2.yml")

In [40]:
# create detector and set parameters
detector = aruco.MarkerDetector()
params = detector.getParams()

In [41]:
# load eye tracking data
eye_data = []
data_file_object  = open("Alison6_15RawData.txt", "r")
data_file_object.readline()
for line in data_file_object.readlines():
    eye_data.append(map(float, line.split()))
data_file_object.close()

In [42]:
# load Blink log
blink_data = []
data_file_object  = open("blinkTimeStamps.txt", "r")
for line in data_file_object.readlines():
    blink_data.append(float(line))
data_file_object.close()

In [43]:
# load saccade log
#start time, end time, duration
saccad_data = []
data_file_object  = open("saccadeData.txt", "r")
data_file_object.readline()
for line in data_file_object.readlines():
    saccad_data.append(map(float, line.split()))
data_file_object.close()

In [44]:
# load visual intake data
visual_intake_data = []
data_file_object = open("visualIntakeData.txt", "r")
data_file_object.readline()
for line in data_file_object.readlines():
    visual_intake_data.append(map(float, line.split()))
data_file_object.close()

In [45]:
# load video

lastStatus =  1 # was the last record inside or oustide? 0 = inside, 1 = outside
currentSessionFrameCount = 0 # to time sessions
sessionMinimum = 2 # minimum frame count to count as session (for smoothing)
sessions = []

cap = cv2.VideoCapture('Alison6_15-1-recording.avi')
ret, frame = cap.read()
    
if not ret:
    print("can't open video!")
    sys.exit(-1)
    
time = cap.get(CV_CAP_PROP_POS_MSEC)

while ret and int(time/16.3) < len(eye_data):
    #print "Time:", int(time), "ms"
    markers = detector.detect(frame)

    if len(markers) == 0: # if no markers detected, assume state is whatever was most recent
        currentSessionFrameCount += 1
        # read next frame
        ret, frame = cap.read()
        time = cap.get(CV_CAP_PROP_POS_MSEC)
        continue
    
    insideSquare = True # current status of gaze
    
    for marker in markers:
        # get marker ID and point positions
        #print("Marker: {:d}".format(marker.id))
        if marker.id == 12:
            slopeH = (marker[1][1]-marker[0][1])/(marker[1][0]-marker[0][0])
            slopeV = (marker[3][0]-marker[0][0])/(marker[3][1]-marker[0][1])
            if not ((eye_data[int(time/16.3)][2] <= slopeH*(eye_data[int(time/16.3)][1] - marker[1][0]) + marker[1][1]) and (eye_data[int(time/16.3)][1] <= slopeV*(eye_data[int(time/16.3)][2] - marker[1][1]) + marker[1][0])):
                insideSquare = False
        elif marker.id == 42:
            slopeH = (marker[3][1]-marker[2][1])/(marker[3][0]-marker[2][0])
            slopeV = (marker[2][0]-marker[1][0])/(marker[2][1]-marker[1][1])
            if not ((eye_data[int(time/16.3)][2] >= slopeH*(eye_data[int(time/16.3)][1] - marker[3][0]) + marker[3][1]) and (eye_data[int(time/16.3)][1] >= slopeV*(eye_data[int(time/16.3)][2] - marker[2][1]) + marker[2][0])):
                insideSquare = False
        elif marker.id == 111:
            slopeV = (marker[2][0]-marker[1][0])/(marker[2][1]-marker[1][1])
            if not (eye_data[int(time/16.3)][1] >= slopeV*(eye_data[int(time/16.3)][2] - marker[2][1]) + marker[2][0]):
                insideSquare = False
        elif marker.id == 277:
            slopeV = (marker[3][0]-marker[0][0])/(marker[3][1]-marker[0][1])
            if not (eye_data[int(time/16.3)][1] <= slopeV*(eye_data[int(time/16.3)][2] - marker[3][1]) + marker[3][0]):
                insideSquare = False
        elif marker.id == 347:
            slopeH = (marker[0][1]-marker[1][1])/(marker[0][0]-marker[1][0])
            if not (eye_data[int(time/16.3)][2] <= slopeH*(eye_data[int(time/16.3)][1] - marker[1][0]) + marker[1][1]):
                insideSquare = False
        elif marker.id == 513:
            slopeH = (marker[3][1]-marker[2][1])/(marker[3][0]-marker[2][0])
            slopeV = (marker[0][0]-marker[3][0])/(marker[0][1]-marker[3][1])
            if not ((eye_data[int(time/16.3)][2] >= slopeH*(eye_data[int(time/16.3)][1] - marker[3][0]) + marker[3][1]) and (eye_data[int(time/16.3)][1] <= slopeV*(eye_data[int(time/16.3)][2] - marker[3][1]) + marker[3][0])):
                insideSquare = False
        elif marker.id == 631:
            slopeH = (marker[3][1]-marker[2][1])/(marker[3][0]-marker[2][0])
            if not (eye_data[int(time/16.3)][2] >= slopeH*(eye_data[int(time/16.3)][1] - marker[3][0]) + marker[3][1]):
                insideSquare = False
        elif marker.id == 888:
            slopeH = (marker[0][1]-marker[1][1])/(marker[0][0]-marker[1][0])
            slopeV = (marker[1][0]-marker[2][0])/(marker[1][1]-marker[2][1])
            if not ((eye_data[int(time/16.3)][2] >= slopeH*(eye_data[int(time/16.3)][1] - marker[1][0]) + marker[1][1]) and (eye_data[int(time/16.3)][1] <= slopeV*(eye_data[int(time/16.3)][2] - marker[1][1]) + marker[1][0])):
                insideSquare = False
        else:
            print "Wrong Marker Recognized"
            
    if insideSquare:
        if lastStatus:
            sessions.append(["OUT", currentSessionFrameCount])
            currentSessionFrameCount = 1
        else:
            currentSessionFrameCount += 1      
        lastStatus = 0
    else:
        if lastStatus:
            currentSessionFrameCount += 1
        else:
            sessions.append(["IN", currentSessionFrameCount])
            currentSessionFrameCount = 1
        lastStatus = 1

    # read next frame
    ret, frame = cap.read()
    time = cap.get(CV_CAP_PROP_POS_MSEC)

if currentSessionFrameCount:
    if lastStatus:
        sessions.append(["OUT", currentSessionFrameCount])
    else:
        sessions.append(["IN", currentSessionFrameCount])
        
i=0
while i < len(sessions):
    if sessions[i][1] < sessionMinimum:
        sessions[i-1][1] += sessions[i][1]
        del sessions[i]
        sessions[i-1][1] += sessions[i][1]
        del sessions[i]
    i += 1

In [50]:
# print output

print "Visualization by frame, divided by second:"
printedframes = 0
for session in sessions:
    for _ in xrange(session[1]):
        if session[0] == "IN":
            if printedframes%24 == 0:
                print
                print printedframes/24 + 1,
            print "X",
            printedframes += 1
        else:
            if printedframes%24 == 0:
                print
                print printedframes/24 + 1,
            print "-",
            printedframes += 1
print "\n"
print "Data by session (tab-separated table):"
print
print "Session\tIN/OUT\tDuration(s)\tBlink Count\tBlink Rate(/s)\tSaccade Count\tSaccade Rate(/s)\tSaccade Average Duration(s)\tVisual Intake Average Pupil Diameter(mm)\tVisual Intake Count\tVisual Intake Frequency(/s)\tVisual Intake Average Duration(s)"

blinkIndex = 0
saccadIndex = 0
visualIntakeIndex = 0
blinkTime = 0
saccadTime = 0
visualIntakeTime = 0
sessionCount = 0
for session in sessions:
    sessionCount += 1
    sessionDuration = float(session[1])/24
    blinkCounter = 0
    saccadCounter = 0
    visualIntakeCounter = 0
    
    blinkTime += sessionDuration
    while blinkIndex < len(blink_data) and blink_data[blinkIndex]*0.001 < blinkTime:
        blinkCounter += 1
        blinkIndex += 1
                
    saccadDurations = []
    saccadAvgDuration = 0
    saccadTime += sessionDuration
    while saccadIndex < len(saccad_data) and saccad_data[saccadIndex][1]*0.001 < saccadTime:
        saccadCounter += 1
        saccadDurations.append(saccad_data[saccadIndex][2])
        saccadIndex += 1
    if len(saccadDurations):
        saccadAvgDuration = sum(saccadDurations)/len(saccadDurations)
    
    pupilSizes = []
    visualIntakeDurations = []
    pupilSizeAvg = 0
    visualIntakeAvgDuration = 0
    visualIntakeTime += sessionDuration
    while visualIntakeIndex < len(visual_intake_data) and visual_intake_data[visualIntakeIndex][0]*0.001 < visualIntakeTime:
        visualIntakeCounter += 1
        pupilSizes.append(visual_intake_data[visualIntakeIndex][2])
        visualIntakeDurations.append(visual_intake_data[visualIntakeIndex][1])
        visualIntakeIndex += 1
    if len(pupilSizes):
        pupilSizeAvg = sum(pupilSizes)/len(pupilSizes)
    if len(visualIntakeDurations):
        visualIntakeAvgDuration = sum(visualIntakeDurations)/len(visualIntakeDurations)
        
    print sessionCount, "\t", session[0], "\t", round(sessionDuration, 3), "\t", blinkCounter, "\t", round(blinkCounter/sessionDuration, 3), "\t", saccadCounter, "\t", round(saccadCounter/sessionDuration, 3), "\t", round(saccadAvgDuration, 3), "\t", round(pupilSizeAvg, 3), "\t", visualIntakeCounter, "\t", round(visualIntakeCounter/sessionDuration, 3), "\t", round(visualIntakeAvgDuration, 3) 
    
insideFrameCount = 0
totalFrameCount = 0
for session in sessions:
    if session[0] == "IN":
        insideFrameCount += session[1]
    totalFrameCount += session[1]
print "Proportion of frames where gaze is inside the box:", round(float(insideFrameCount)/(totalFrameCount), 3)

Visualization by frame, divided by second:

1 - - - - - - - - - - - - - - - - - - - - - - - -
2 - - - - - - - - - - - - - - - - - - - - - - - -
3 - - - - - - - - - - - - - - - - - - - - - - - -
4 - - - - X X X X X X X X X X X X X X X X X X X X
5 X X X X X X X X X X X X X X X X X X X X X X X X
6 X X X X X X X X X X X X X X X X X X X X X X X X
7 X X X X X X X X X X X X X X X X X X X X X X X X
8 X X X X X X X X X X X X X X X X X X X X X X X X
9 X X X X X X X X X X X X X X X X X X X X X X X X
10 X X X X X X X X X X X X X X X X X X X X X X X X
11 X X X X X X X X X X X X X X X X X X X X X X X X
12 X X X X X X X X X X X X X X X X X X X X X X X X
13 X X X X X X X X X X X X X X X X X X X X X X X X
14 X X X X X X X X X X X X X X X X X X X X X X X X
15 X X X X X X X X X X X X X X - - - - - - - - - -
16 - - - - - - - - - - - - - - - - - - - - - - X X
17 X X X X X X X X X X X X X X X X X X X X X X X X
18 - - - - - - - - - - - X X X X X X X X X X X X X
19 X X X X X X X X X X X X X X X X X X X X X X 